In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## 1 Load Dataset


In [5]:
df = pd.read_csv('./data/temperature_data.csv')
df.head()

,Day,Temperature
0,1,0.133553
1,2,0.061635
2,3,0.175963
3,4,0.302001
4,5,0.082970


##### Goal of generation:

- perhaps look at first 30 - 60 days of temperature data
- then from pattern, generate for next n days, following timeseries forecasting prediction to generate
- i.e not one-step prediction but range-prediction


In [6]:
df.isna().sum()

Day            0
Temperature    0
dtype: int64

In [7]:
temp_data = df['Temperature'].values

### 1.1 Data Preprocessing (Time-series)

- specify custom dataset (prep time-series data for training)
- efficient data handling
- compatible with data loaders
- customized preprocessing
  - converting data to tensors

### Usage steps of Custom Dataset class

1. pass temperature data to custom dataset for sequence to be created
2. create dataloader with help of custom dataset
3. feed dataloader to model


In [17]:
T = 30 # sequence length
batch_size = 32 # No. of sequences to be trained per iteration

# Dataset -> handling, managing data in structured way
class TimeSeriesDataset(Dataset): 
    def __init__(self, data, sequence_length): # init custom dataset
        self.data = data # to hold the temperature data
        self.sequence_length = sequence_length # No. time steps to predict next value
    
    # return total No. of sequence that can be created, given no overlap
    # eg: total data = 3, sequence length = 2, then we have 3 - 2 = 1 (1,2) or (2,3)
    # +1 if overlap allowed
    def __len__(self):  
        return len(self.data) - self.sequence_length

    # retrieve specific sequence & corresponding target value
    # i is start idx
    def __getitem__(self, i):
        x = self.data[i: i+self.sequence_length] # sequence data
        y = self.data[i+self.sequence_length] # target
        
        x = torch.tensor(x, dtype=torch.float32).unsqueeze(-1) # batch_size x sequence_length x input_dim
        y = torch.tensor(y, dtype=torch.float32)

        return x, y


In [18]:
temp_dataset = TimeSeriesDataset(temp_data, T) # T = sequence_length
temp_dataloader = DataLoader(temp_dataset, batch_size=batch_size, shuffle=False) # time-series should not shuffle data, order is impt

## 2 Generator / Discriminator Model Init

- the design of the Neural network would be different for time-series data
- time-series data = LSTM


In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim, output_dim, seq_len): # z_dim is the input dim
        super(Generator, self).__init__() # init nn.Module
        self.lstm = nn.LSTM(z_dim, hidden_dim, num_layers=2, batch_first=True) # batch_first means (batchsize x seq_length x hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim) # adding linear layer + activation after can help increase non-linearity and make model capture more complex r/s
    
    def forward(self, x): # pass data through the layers created
        h_0 = torch.zeros(2, x.size(0), hidden_dim) # to store all hidden states for LSTM (2 hidden layers, batchsize, hidden_dim)
        c_0 = torch.zeros(2, x.size(0), hidden_dim) # to store all cell states for LSTM
        
        lstm_out, _ = self.lstm(x, (h_0, c_0)) # _ represents the final hidden and cell state output
        out = self.fc(lstm_out)
        return out